In [ ]:
!pip install simpy

a coffee machine needs refills at random times(by 1 worker)

the worker must clean the floors, less important tasks as well

coffee refill preempts the cleaning task

the cleaning continues after done with refilling the coffee machine

### approach 1 (w/o preemptive res)

In [ ]:
import simpy, random

def cafe_machine(env):
  while True:
    print(f'Cafe machine ready to use at {env.now:.2f}')
    rand_inter_time = 100 #random.uniform(60, 180)
    yield env.timeout(rand_inter_time)
    print(f'Cafe machine done. Call worker at {env.now:.2f}')
    cleaning.interrupt()
    global refilled_ev
    yield refilled_ev
    refilled_ev = env.event()

def clean_floor(env, worker):
  time_to_clean = 120
  while True:
    try:
      while time_to_clean:
        print(f'Cleaning started at {env.now:.2f} and time to clean = {time_to_clean:.2f}')
        with worker.request() as req:
          yield req
          start = env.now
          yield env.timeout(time_to_clean)
          time_to_clean = 120
    except simpy.Interrupt:
      current_time = env.now
      time_to_clean -= current_time - start
      if time_to_clean == 0:
        time_to_clean = 120
      print(f'Cleaning interrupted at {env.now:.2f} and time left to clean = {time_to_clean:.2f}')
      yield env.process(refilling(env, worker))



def refilling(env, worker):
  with worker.request() as req:
    yield req
    print(f'Refilling machine started at {env.now:.2f}')
    yield env.timeout(15)
    print(f'Refilled machine is ready to use at {env.now:.2f}')
    global refilled_ev
    refilled_ev.succeed()

env = simpy.Environment()
refilled_ev = env.event()
worker = simpy.Resource(env, capacity=1)

worker = simpy.Resource(env, capacity=1)
env.process(cafe_machine(env))
cleaning = env.process(clean_floor(env, worker))
env.run(until=24*60)

### 2 (w/ preemptive res)

In [ ]:
import simpy, random

def cafe_machine(env):
  while True:
    print(f'Cafe machine ready to use at {env.now:.2f}')
    rand_inter_time = 100 #random.uniform(60, 180)
    yield env.timeout(rand_inter_time)
    print(f'Cafe machine done. Call worker at {env.now:.2f}')
    yield env.process(refilling(env, worker))

def clean_floor(env, worker):
  time_to_clean = 120
  while True:
    try:
      while time_to_clean:
        print(f'Cleaning started at {env.now:.2f} and time to clean = {time_to_clean:.2f}')
        with worker.request(priority=2) as req:
          yield req
          start = env.now
          yield env.timeout(time_to_clean)
          print(f'Cleaning finishedat time {env.now:.2f} and time to clean = {time_to_clean:.2f}')
          time_to_clean = 120
    except simpy.Interrupt:
      current_time = env.now
      time_to_clean -= current_time - start
      if time_to_clean == 0:
        time_to_clean = 120
      print(f'Cleaning interrupted at {env.now:.2f} and time left to clean = {time_to_clean:.2f}')


def refilling(env, worker):
  with worker.request(priority=1) as req:
    yield req
    print(f'Refilling machine started at {env.now:.2f}')
    yield env.timeout(15)
    print(f'Refilled machine is ready to use at {env.now:.2f}')

env = simpy.Environment()
worker = simpy.PreemptiveResource(env, capacity=1)
env.process(cafe_machine(env))
cleaning = env.process(clean_floor(env, worker))
env.run(until=24*60)

Cafe machine ready to use at 0.00
Cleaning started at 0.00 and time to clean = 120.00
Cafe machine done. Call worker at 100.00
Cleaning interrupted at 100.00 and time left to clean = 20.00
Cleaning started at 100.00 and time to clean = 20.00
Refilling machine started at 100.00
Refilled machine is ready to use at 115.00
Cafe machine ready to use at 115.00
Cleaning finishedat time 135.00 and time to clean = 20.00
Cleaning started at 135.00 and time to clean = 120.00
Cafe machine done. Call worker at 215.00
Cleaning interrupted at 215.00 and time left to clean = 40.00
Cleaning started at 215.00 and time to clean = 40.00
Refilling machine started at 215.00
Refilled machine is ready to use at 230.00
Cafe machine ready to use at 230.00
Cleaning finishedat time 270.00 and time to clean = 40.00
Cleaning started at 270.00 and time to clean = 120.00
Cafe machine done. Call worker at 330.00
Cleaning interrupted at 330.00 and time left to clean = 60.00
Cleaning started at 330.00 and time to clean 